## 使用 `czsc` 进行选股
---


In [1]:
import sys
sys.path.insert(0, ".")
sys.path.insert(0, "..")

import czsc
print(czsc.__version__)

from datetime import datetime
from typing import List
import traceback
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from czsc.analyze import KlineAnalyze

0.5.4


In [2]:
# 导入 Tushare 数据（推荐使用）
from czsc.data.ts import *

# 如果是第一次使用需要设置 token
# set_token("trushare.pro token")

In [ ]:
# 导入聚宽数据
from czsc.data.jq import *

# 如果是第一次使用需要设置 token
# set_token("手机号", "密码")

### 缠论日线笔中枢三买选股
---

条件描述： 最近五笔走势，前三笔构成中枢，第四笔离开中枢，第五笔向下不回中枢



In [3]:
def selector(symbols: List):
    """输入股票列表，输入符合买点定义的股票"""
    res = []
    for symbol in tqdm(symbols, desc="缠论日线笔中枢三买选股"):
        try:
            kline = get_kline(symbol=symbol, end_date=datetime.now(), freq="D", count=1000)
            ka = KlineAnalyze(kline, ma_params=(5, 34, 120, 233), bi_mode="new")
            points = ka.bi_list[-6:]
            
            if len(points) == 6 and points[-1]['fx_mark'] == "d":
                zs_g = min([x['bi'] for x in points[:4] if x['fx_mark'] == 'g'])
                zs_d = max([x['bi'] for x in points[:4] if x['fx_mark'] == 'd'])
                
                if points[-1]['bi'] > zs_g > zs_d:
                    res.append(symbol)
                    
        except:
            print("{} 分析失败".format(symbol))
            traceback.print_exc()
    return res

### 日线笔中枢选股
---

中枢如果能当下确认，基本逻辑如下：
1. ma 233<close<ma 60 ---得 codes_list1
2. codes_list1中选择 符合日线笔中枢形态的，得codes_list2
    * 2.1 笔中枢形态：方向下: 日线中枢完成，第四笔底分型
3. codes_list2 中中枢第一笔到第四笔得日期跨度 date1
4. date1时间跨度中，codes_list2 ma233 cross的个数，并标注 codes_list3

In [4]:
def cross_number(x1, x2):
    """输入两个序列，计算 x1 下穿 x2 的次数"""
    x = np.array(x1) < np.array(x2)
    num = 0
    for i in range(len(x)-1):
        b1, b2 = x[i], x[i+1]
        if b2 and b1 != b2:
            num += 1
    return num

x1 = [1, 1, 3, 4, 5, 12, 9, 8]
x2 = [2, 2, 1, 5, 8, 9, 10, 3]
print(cross_number(x1, x2))

2


In [5]:
def selector(symbols: List):
    """输入股票列表，输入符合买点定义的股票"""
    res = []
    for symbol in tqdm(symbols, desc="缠论选股"):
        try:
            kline = get_kline(symbol=symbol, end_date=datetime.now(), freq="D", count=1000)
            ka = KlineAnalyze(kline, ma_params=(5, 34, 60, 250), bi_mode="new")

            if ka.ma[-1]['ma60'] >= ka.latest_price >= ka.ma[-1]['ma250']:
                # print("{} 满足条件1：ma60 > close > ma233".format(symbol))
                points = ka.bi_list[-7:]
                
                if len(points) == 7 and points[-1]['fx_mark'] == 'd':
                    zs_g = min([x['bi'] for x in points[2:6] if x['fx_mark'] == 'g'])
                    zs_d = max([x['bi'] for x in points[2:6] if x['fx_mark'] == 'd'])
                    
                    if zs_g > zs_d:
                        # print("{} 满足条件2：向下中枢完成".format(symbol))
                        date_span = [points[-5]['dt'], points[-1]['dt']]
                        low = [x['low'] for x in ka.kline_raw if date_span[1] >= x['dt'] >= date_span[0]]
                        ma_ = [x['ma250'] for x in ka.ma if date_span[1] >= x['dt'] >= date_span[0]]
                        num = cross_number(low, ma_)
                        res.append({"symbol": symbol, "cross_num": num})
        except:
            print("{} 分析失败".format(symbol))
            traceback.print_exc()
    return res

## 执行选股
---

In [6]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("399102.SZ")
selected = selector(symbols)

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)

..\czsc\data\ts.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k[col] = k[col].apply(round, args=(2,))


300793.SZ-E 分析失败


Traceback (most recent call last):
  File "<ipython-input-5-14c245f5b689>", line 7, in selector
    ka = KlineAnalyze(kline, ma_params=(5, 34, 60, 250), bi_mode="new")
  File "..\czsc\analyze.py", line 327, in __init__
    self._update_xd_list()
  File "..\czsc\analyze.py", line 633, in _update_xd_list
    self._xd_after_process()
  File "..\czsc\analyze.py", line 627, in _xd_after_process
    if (self.xd_list[-1]['fx_mark'] == 'd' and self.bi_list[-1]['bi'] < self.xd_list[-1]['xd']) \
IndexError: list index out of range



选股结果： [{'symbol': '300246.SZ-E', 'cross_num': 0}, {'symbol': '300206.SZ-E', 'cross_num': 0}, {'symbol': '300122.SZ-E', 'cross_num': 0}, {'symbol': '300573.SZ-E', 'cross_num': 0}, {'symbol': '300530.SZ-E', 'cross_num': 0}, {'symbol': '300529.SZ-E', 'cross_num': 0}, {'symbol': '300496.SZ-E', 'cross_num': 0}, {'symbol': '300489.SZ-E', 'cross_num': 0}, {'symbol': '300750.SZ-E', 'cross_num': 0}, {'symbol': '300699.SZ-E', 'cross_num': 0}, {'symbol': '300677.SZ-E', 'cross_num': 0}, {'symbol': '300661.SZ-E', 'cross_num': 0}, {'symbol': '300635.SZ-E', 'cross_num': 0}, {'symbol': '300624.SZ-E', 'cross_num': 0}, {'symbol': '300618.SZ-E', 'cross_num': 0}, {'symbol': '300601.SZ-E', 'cross_num': 0}, {'symbol': '300595.SZ-E', 'cross_num': 0}, {'symbol': '300454.SZ-E', 'cross_num': 0}, {'symbol': '300014.SZ-E', 'cross_num': 0}]


In [ ]:
# 使用聚宽数据在创业板综指上选股
symbols = get_index_stocks("399102.XSHE")
selected = selector(symbols)

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)